In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np

In [2]:
categories = ['EDUCATION', 'BUSINESS', 
       'DATING', 'SPORTS', 
       'WEATHER', 'FOOD_AND_DRINK', 
       'HEALTH_AND_FITNESS', 'BEAUTY', 
       'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES', 
       'SOCIAL', 'SHOPPING', 
       'PRODUCTIVITY', 'PHOTOGRAPHY', 
       'MEDICAL', 'PARENTING', 
       'EDUCATION', 'BEAUTY',
       'COMMUNICATION', 'TOOLS']

In [ ]:
def get_links(data):
    '''Given html data this will extract all the links'''
    link_list = []
    for link in data.findAll('a'):
        get_link = link.get('href')
        link_list.append(get_link)
    return link_list

In [ ]:
def webpage_html(url):
    r = requests.get(url)
    html = r.text
    return html

In [ ]:
def google_scraper(categories):
    '''Will cycle through every category and then request the site, pass the soup to 
        get_links and then clean the list and append results to a new list in a dict
        format that has the format of {cat_name : [link list]}'''
    dicty = dict()
    for category in categories:
        url = f'https://play.google.com/store/apps/category/{category}/collection/topselling_free'
        html = webpage_html(url)
        soup = BeautifulSoup(html)
        links = get_links(soup)
        del links[0:107]
        clean = list(set(links))
        new_list = [app for app in clean if app.find('details') != -1]
        dicty[category] = new_list
        time.sleep(3)
    return dicty


In [3]:
cat = categories

In [ ]:
stuff = google_scraper(cat)

In [ ]:
stuff

In [ ]:
x = 0
for key, value in stuff.items():
    x+=len(value)
x


In [ ]:
def parse(listing_html, criteria):
    result = listing_html.find(class_=re.compile(r'.*%s' % criteria))
    if result:
        return result.text

In [ ]:
def get_description(category_dict):
    dicty = dict()
    url = 'https://play.google.com'
    for category, link_list in category_dict.items():
        listy = list()
        for link in link_list:
            html = webpage_html(url+link)
            soup = BeautifulSoup(html)
            html1 = soup.find('div', {'class':"LXrl4c"})
            description = parse(html1, 'DWPxHb')
            time.sleep(1)
            listy.append(description)
        dicty[category] = listy
    return dicty              

In [ ]:
all_desc = get_description(stuff)

In [2]:
# # Save
# np.save('my_file.npy', all_desc) 

# # # Load
read_dictionary = np.load('my_file.npy').item()

In [ ]:
x = 0
y = 0
for k,v in all_desc.items():
    x+= len(v)
    y += 1
y, x

In [3]:
edu  = read_dictionary

In [4]:
read_dictionary.keys()

dict_keys(['EDUCATION', 'BUSINESS', 'DATING', 'SPORTS', 'WEATHER', 'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'BEAUTY', 'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES', 'SOCIAL', 'SHOPPING', 'PRODUCTIVITY', 'PHOTOGRAPHY', 'MEDICAL', 'PARENTING', 'COMMUNICATION', 'TOOLS'])

In [5]:
import nltk
import sklearn

from nltk.collocations import *
from nltk import FreqDist, word_tokenize
import string, re
from nltk.stem.snowball import SnowballStemmer

In [6]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"

In [7]:
# stop words
from nltk.corpus import stopwords
stopwords.words("english")

stop_words = set(stopwords.words('english'))

In [8]:
# stem words
stemmer = SnowballStemmer("english")

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [10]:
review_list = []
for c, d in edu.items():
    for review in d:
        art_tokens_raw = nltk.regexp_tokenize(review, pattern)
        art_tokens = [i.lower() for i in art_tokens_raw]
        art_tokens_stopped = [w for w in art_tokens if not w in stop_words]
        art_stemmed = [stemmer.stem(word) for word in art_tokens_stopped]
        cleaned = ' '.join(art_stemmed)
        review_list.append(cleaned)

In [11]:
response = tfidf.fit_transform(review_list )

df = pd.DataFrame(response.toarray(), columns=tfidf.get_feature_names())


In [12]:
x = read_dictionary.keys()
new_list = []
for c in x: 
    s = [c] * 60
    new_list += s

In [13]:
len(new_list)

1080

In [14]:
df.shape

(1080, 13205)

In [15]:
df['labels'] = new_list

In [16]:
df.tail()

,aa,aac,aaptiv,aarp,aask,ab,abajo,abandon,abc,abcmous,...,zoomterrain,zoomwithushav,zoosk,zte,zulili,zulu,zulufor,zulupermiss,zumba,labels
1075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOOLS
1076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOOLS
1077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOOLS
1078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOOLS
1079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOOLS


# Test-Train Split

In [17]:
y = df.labels

In [18]:
X = df.iloc[:,:-1]

In [28]:
# Sample code
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

In [29]:
from tpot import TPOTClassifier

In [45]:
tpot = TPOTClassifier(generations=2,scoring= 'accuracy' ,cv = 3 ,population_size=5,\
                      verbosity=3, warm_start = True)

In [46]:
tpot.fit(X_train, y_train)

30 operators have been imported by TPOT.


Skipped pipeline #3 due to time out. Continuing to the next pipeline.
Skipped pipeline #5 due to time out. Continuing to the next pipeline.
Skipped pipeline #9 due to time out. Continuing to the next pipeline.
Generation 1 - Current Pareto front scores:
-1	0.7373086881440823	LinearSVC(input_matrix, LinearSVC__C=5.0, LinearSVC__dual=True, LinearSVC__loss=hinge, LinearSVC__penalty=l2, LinearSVC__tol=1e-05)

Skipped pipeline #15 due to time out. Continuing to the next pipeline.
Skipped pipeline #17 due to time out. Continuing to the next pipeline.
Skipped pipeline #20 due to time out. Continuing to the next pipeline.
Generation 2 - Current Pareto front scores:
-1	0.7429858695627892	LinearSVC(input_matrix, LinearSVC__C=1.0, LinearSVC__dual=True, LinearSVC__loss=hinge, LinearSVC__penalty=l2, LinearSVC__tol=1e-05)



TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=3,
        disable_update_check=False, early_stop=None, generations=2,
        max_eval_time_mins=5, max_time_mins=None, memory=None,
        mutation_rate=0.9, n_jobs=1, offspring_size=None,
        periodic_checkpoint_folder=None, population_size=5,
        random_state=None, scoring='accuracy', subsample=1.0,
        use_dask=False, verbosity=3, warm_start=True)

In [55]:
tpot.score(X_test, y_test)

ValueError: could not convert string to float: 'PHOTOGRAPHY'

In [48]:
tpot.export('class2-pipeline.py')

True

In [ ]:
# 0.8009259259259259 First pipeline trial

In [49]:
exported_pipeline = LinearSVC(C=1.0, dual=True, loss="hinge", penalty="l2", tol=1e-05)

In [51]:
exported_pipeline.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=1e-05, verbose=0)

In [53]:
exported_pipeline.score(X_test, y_test)

0.8055555555555556